In [1]:
import pandas as pd
import openpyxl
import csv
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
# from celery import Celery
from selenium import webdriver
from selenium.webdriver.support.ui import Select
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException 
import re
import time
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.alert import Alert
import geocoder
from tqdm import tqdm
import random
import glob
import itertools

## 01 批次抓取河川水位的網頁資料

1.製作3個年度的清單與編號
2.放入查詢網站查詢
3.(取消)製作爬取時的時間標記

#### 製作2019-2023年度的清單與編號

In [2]:
# 起始日期
start_date = datetime(2023, 3, 1)

# 結束日期
end_date = datetime(2023, 10, 31)

# 設定步長為一天
step = timedelta(days=1)

# 生成日期範圍
date_range = [start_date + i * step for i in range((end_date - start_date).days // step.days + 1)]

# 顯示生成的日期
temp = []
for date in date_range:
    x = date.strftime('%Y/%m/%d')
    temp.append(x)

final_datelist = pd.DataFrame({"date": temp})
# final_datelist = final_datelist.iloc[::-1]
# final_datelist = final_datelist.reset_index(drop=True)
print(final_datelist)

           date
0    2023/03/01
1    2023/03/02
2    2023/03/03
3    2023/03/04
4    2023/03/05
..          ...
240  2023/10/27
241  2023/10/28
242  2023/10/29
243  2023/10/30
244  2023/10/31

[245 rows x 1 columns]


In [3]:
# 生成从00:00到23:00的整点时间列表
# hour_list = ["{:02d}:00".format(hour) for hour in range(24)]

# 時間過多，只取00:00, 06:00, 12:00, 18:00
hour_list = ['00:00', '06:00', '12:00', '18:00']
print(hour_list)

['00:00', '06:00', '12:00', '18:00']


#### 爬蟲部分

In [6]:
for i in range(len(final_datelist)):
    year = final_datelist.loc[i, "date"].split("/")[0]
    month = final_datelist.loc[i, "date"].split("/")[1]
    day = int(final_datelist.loc[i, "date"].split("/")[2]) - 1
    for j in range(len(hour_list)):
        hour = hour_list[j]
        print(year, month, day, hour)


2023 01 0 00:00
2023 01 0 06:00
2023 01 0 12:00
2023 01 0 18:00
2023 01 1 00:00
2023 01 1 06:00
2023 01 1 12:00
2023 01 1 18:00
2023 01 2 00:00
2023 01 2 06:00
2023 01 2 12:00
2023 01 2 18:00
2023 01 3 00:00
2023 01 3 06:00
2023 01 3 12:00
2023 01 3 18:00
2023 01 4 00:00
2023 01 4 06:00
2023 01 4 12:00
2023 01 4 18:00
2023 01 5 00:00
2023 01 5 06:00
2023 01 5 12:00
2023 01 5 18:00
2023 01 6 00:00
2023 01 6 06:00
2023 01 6 12:00
2023 01 6 18:00
2023 01 7 00:00
2023 01 7 06:00
2023 01 7 12:00
2023 01 7 18:00
2023 01 8 00:00
2023 01 8 06:00
2023 01 8 12:00
2023 01 8 18:00
2023 01 9 00:00
2023 01 9 06:00
2023 01 9 12:00
2023 01 9 18:00
2023 01 10 00:00
2023 01 10 06:00
2023 01 10 12:00
2023 01 10 18:00
2023 01 11 00:00
2023 01 11 06:00
2023 01 11 12:00
2023 01 11 18:00
2023 01 12 00:00
2023 01 12 06:00
2023 01 12 12:00
2023 01 12 18:00
2023 01 13 00:00
2023 01 13 06:00
2023 01 13 12:00
2023 01 13 18:00
2023 01 14 00:00
2023 01 14 06:00
2023 01 14 12:00
2023 01 14 18:00
2023 01 15 00:00
202

#### Crawler function

In [4]:
def crawler_data(year, month, day, hour):
    random_wait_time = random.uniform(1, 3)
    driver.get(url)
    # time.sleep(10)

    # try:
    #     alert = driver.switch_to.alert
    #     alert.accept()  # 或者使用 alert.dismiss()，視情況而定
    # except:
    #     print("未找到警告窗口")

    button_entry = driver.find_element_by_id("hour-input")
    button_entry.click()
    button_entry.clear()
    button_entry.send_keys(hour)
    time.sleep(random_wait_time)

    # 找到查詢建照的button
    button_entry = driver.find_element_by_id("userDate")
    button_entry.click()
    # button_entry.clear()
    time.sleep(random_wait_time)
    # button_entry.send_keys(str(final_datelist['date'][0]))

    # 定位下拉式清單元素
    select_element = driver.find_element_by_class_name("ui-datepicker-year")
    # 使用 Select 類初始化
    select = Select(select_element)
    # 選擇指定的年
    select.select_by_value(year)
    time.sleep(1)

    # 定位月份下拉式清單元素
    select_element = driver.find_element_by_class_name("ui-datepicker-month")
    # 使用 Select 類初始化
    select = Select(select_element)
    # 選擇指定的選項，這裡選擇 1 月 (索引值從 0 開始)
    select.select_by_value(str(month))
    time.sleep(1)

    # 找到所有日期元素
    date_elements = driver.find_elements_by_class_name("ui-state-default")
    # 選擇要點擊的日期元素，這裡選擇第一個元素
    desired_date_element = date_elements[day]  # 選擇第一個日期元素
    # 點擊指定的日期元素
    desired_date_element.click()
    time.sleep(1)

    button_entry = driver.find_element_by_id("date-select")
    time.sleep(random_wait_time)
    button_entry.click()
    time.sleep(random_wait_time)

    # try:
    #     alert = driver.switch_to.alert
    #     alert.accept()  # 或者使用 alert.dismiss()，視情況而定
    # except:
    #     print("未找到警告窗口")

    # 找到包含所有資料的 <table> 元素
    table = driver.find_element_by_xpath("//table[@class='table table-condensed table-striped']")

    # 找到所有的 <tr> 元素
    rows = table.find_elements_by_xpath(".//tbody/tr")

    # 用於存儲資料的列表
    data = []

    # 遍歷每一行
    for row in rows:
        # 找到這一行中所有的 <td> 元素
        cells = row.find_elements_by_tag_name("td")
        # 提取每個 <td> 元素的文本內容並存儲在 data 中
        row_data = [cell.text for cell in cells]
        data.append(row_data)
    
    final_df = pd.DataFrame(data)
    final_df['datetime'] = final_datelist['date'][i] + " " + str(hour)
    
    # Create file datetime
    year_s = final_datelist.loc[i, "date"].split("/")[0]
    month_s = final_datelist.loc[i, "date"].split("/")[1]
    day_s = final_datelist.loc[i, "date"].split("/")[2]
    hour_s = hour.split(":")[0]
    name = year_s + month_s + day_s + hour_s
    path = "D:/Chu's Document!/02 Project/06 道路塌陷防治專案(天坑)/03 Data/Crawled/River_level/" + str(name) + ".csv"
    final_df.to_csv(path, index=False)
    driver.close()

In [5]:
# 生成日期和小时的组合
date_hour_combinations = itertools.product(final_datelist.iterrows(), hour_list)

log = []

for (i, date_row), hour in date_hour_combinations:

    # 訪問網頁
    url = "http://211.23.167.210/powerrefor/riverWarning/riverWarningMap10RB.html"
    chrome_path = r"C:\Users\doit\anaconda3\envs\practice_02\chromedriver-win64\chromedriver.exe"
    driver = webdriver.Chrome(executable_path=chrome_path)
    driver.implicitly_wait(15)

    # Extract input
    year = date_row["date"].split("/")[0]
    month = int(date_row["date"].split("/")[1]) - 1
    day = int(date_row["date"].split("/")[2]) - 1    

    # Create file datetime
    year_s = final_datelist.loc[i, "date"].split("/")[0]
    month_s = final_datelist.loc[i, "date"].split("/")[1]
    day_s = final_datelist.loc[i, "date"].split("/")[2]
    hour_s = hour.split(":")[0]
    name = year_s + month_s + day_s + hour_s

    try:
        # 在 try 块中调用 crawl_data 函数
        crawler_data(year, month, day, hour)
        log.append({"datetime": name, "status": "Success"})

        print(f"已完成 {len(log)}/{len(final_datelist)*len(hour_list)} 筆資料的爬取")

    except Exception as e:
        print(e)
        log.append({"datetime": name, "status": "Failed"})
        driver.quit()

# 将日志转换为 DataFrame，并保存为 CSV 文件
log_df = pd.DataFrame(log)
log_df.to_csv("D:/Chu's Document!/02 Project/06 道路塌陷防治專案(天坑)/03 Data/Crawled/River_level/log.csv", index=False)
print("日志已保存到文件")

已完成 1/980 筆資料的爬取
已完成 2/980 筆資料的爬取
已完成 3/980 筆資料的爬取
已完成 4/980 筆資料的爬取
已完成 5/980 筆資料的爬取
已完成 6/980 筆資料的爬取
已完成 7/980 筆資料的爬取
已完成 8/980 筆資料的爬取
已完成 9/980 筆資料的爬取
已完成 10/980 筆資料的爬取
已完成 11/980 筆資料的爬取
已完成 12/980 筆資料的爬取
已完成 13/980 筆資料的爬取
已完成 14/980 筆資料的爬取
已完成 15/980 筆資料的爬取
已完成 16/980 筆資料的爬取
已完成 17/980 筆資料的爬取
已完成 18/980 筆資料的爬取
已完成 19/980 筆資料的爬取
已完成 20/980 筆資料的爬取
已完成 21/980 筆資料的爬取
已完成 22/980 筆資料的爬取
已完成 23/980 筆資料的爬取
已完成 24/980 筆資料的爬取
已完成 25/980 筆資料的爬取
已完成 26/980 筆資料的爬取
已完成 27/980 筆資料的爬取
已完成 28/980 筆資料的爬取
已完成 29/980 筆資料的爬取
已完成 30/980 筆資料的爬取
已完成 31/980 筆資料的爬取
已完成 32/980 筆資料的爬取
已完成 33/980 筆資料的爬取
已完成 34/980 筆資料的爬取
已完成 35/980 筆資料的爬取
已完成 36/980 筆資料的爬取
已完成 37/980 筆資料的爬取
已完成 38/980 筆資料的爬取
已完成 39/980 筆資料的爬取
已完成 40/980 筆資料的爬取
已完成 41/980 筆資料的爬取
已完成 42/980 筆資料的爬取
已完成 43/980 筆資料的爬取
已完成 44/980 筆資料的爬取
已完成 45/980 筆資料的爬取
已完成 46/980 筆資料的爬取
已完成 47/980 筆資料的爬取
已完成 48/980 筆資料的爬取
已完成 49/980 筆資料的爬取
已完成 50/980 筆資料的爬取
已完成 51/980 筆資料的爬取
已完成 52/980 筆資料的爬取
已完成 53/980 筆資料的爬取
已完成 54/980 筆資料的爬取
已完成 55/980 筆資料的爬取
已完成 56/980 筆資料的爬取
已

In [126]:
# 找到查詢建照的button
button_entry = driver.find_element_by_id("userDate")
# button_entry.click()
button_entry.send_keys(str(final_datelist[0]))

In [47]:
crawler_save.tail()
# print(crawler_save.shape)

,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名
99,108建0137號,109使0031號,,108/11/26,地上2層，地下0層,2層 1戶,大同區圓環段三小段0560-0022等4筆,103大同區103臺北市大同區星明里寧夏路27號,財團法人陳德星堂 陳春銅
100,108建0138號,112使0016號,109/07/07,111/02/21,地上4層，地下2層,6層 4戶,內湖區康寧段一小段0424-0000等1筆,114內湖區114臺北市內湖區秀湖里大湖街166巷44號,高楊碧菁
101,108建0139號,111使0051號,109/05/04,110/10/19,地上2層，地下0層,2層 1戶,內湖區西湖段二小段0037-0000等1筆,114內湖區114臺北市內湖區西安里內湖路一段91巷229號（旁）,許金蘭
102,108建0140號,,109/05/12,,地上24層，地下4層,28層 290戶,松山區寶清段一小段0057-0013等1筆,105松山區105臺北市松山區自強里延壽街,中華工程股份有限公司 代表人：朱蕙蘭
103,108建0141號,112使0013號,109/05/07,111/11/14,地上15層，地下2層,17層 102戶,中山區德惠段二小段0813-0001等6筆,104中山區104臺北市中山區新福里民權東路二段,昇陽建設企業股份有限公司 負責人：麥寬成


In [4]:
crawler_save.to_excel(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Crawled\crawler_save_106-108_07.xlsx")

## 02 處理河川水位資料

#### 導入與去除重複值 (僅讀入2個檔案)

In [69]:
# 儲存成excel與CSV
merged_data.to_csv(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\106-112建照.csv")
merged_data.to_excel(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\106-112建照.xlsx")
print("done")

done
